In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16495,2024-03-20,República Tcheca Nbl,14:00,Nymburk,USK Praga,1.01,13.00,150.5,1.88,1.79,0.0,0.00,0.00,Ac0qYbED,0.990099,0.076923,0.531915,0.558659,0.067022,101.474,10.686018,0.105308,2.4,1.341641,0.559017,101.60,1.144,0.084439,0.073811,54.0,181.972,82.860656,0.455348,0.0,0.000000,NaN,215.15,2.836,1.155846,0.407562,-105.0,80,65,1.27,3.31,102.242,260.332,1.210308,0.034681,NaN,0.49,0.098,0.102041,0.000000,0.0,0.000000,-5.00,-1.000,-12.000000,0.349037,0.0,-0.349037
16496,2024-03-20,Polônia 1. Liga,15:00,Gornik Walbrzych,Alstom Krakow,1.03,10.10,160.5,1.83,1.83,0.0,0.00,0.00,d0XTEUVK,0.970874,0.099010,0.546448,0.546448,0.069884,106.868,13.235166,0.123846,1.2,1.643168,1.369306,112.59,1.272,0.143944,0.113164,10.0,245.896,92.893115,0.377774,0.6,1.341641,2.236068,334.56,3.084,1.045194,0.338908,-66.0,81,82,1.39,4.08,103.254,244.826,1.152463,0.000000,NaN,-1.57,-0.314,-0.095541,0.840008,0.7,-0.140008,-5.00,-1.000,-9.100000,0.290655,0.1,-0.190655
16497,2024-03-20,República Tcheca Nbl,13:30,Pardubice,Kolin,1.14,5.19,162.5,1.87,1.79,0.0,0.00,0.00,lrCzzLq1,0.877193,0.192678,0.534759,0.558659,0.069871,204.530,67.552572,0.330282,3.0,0.000000,0.000000,320.38,2.490,0.799906,0.321247,77.0,313.932,171.317409,0.545715,0.6,1.341641,2.236068,255.64,3.962,2.170811,0.547908,-67.0,83,83,3.86,3.08,147.240,380.680,0.904829,0.030912,NaN,0.33,0.066,2.121212,0.590125,0.5,-0.090125,-5.00,-1.000,-4.190000,0.311410,0.2,-0.111410
16498,2024-03-20,República Tcheca Nbl,14:00,Decin,Usti n. Labem,1.63,2.19,165.5,1.87,1.79,0.0,0.00,0.00,W61uZva7,0.613497,0.456621,0.534759,0.558659,0.070118,186.968,121.578723,0.650265,1.8,1.643168,0.912871,146.06,2.186,1.395647,0.638448,-5.0,173.634,76.644905,0.441416,2.4,1.341641,0.559017,307.11,2.030,0.929328,0.457797,33.0,109,87,1.34,3.53,115.686,201.578,0.207319,0.030912,NaN,0.53,0.106,5.943396,0.688429,0.8,0.111571,-2.34,-0.468,-2.542735,0.613169,0.5,-0.113169
16499,2024-03-20,República Tcheca Nbl,14:00,Opava,Brno,1.20,4.27,163.5,1.85,1.85,0.0,0.00,0.00,zu8Wz1Uf,0.833333,0.234192,0.540541,0.540541,0.067525,191.312,141.929793,0.741876,0.6,1.341641,2.236068,94.71,2.374,1.823631,0.768168,-13.0,171.922,117.096681,0.681104,1.8,1.643168,0.912871,106.60,2.010,1.381901,0.687513,65.0,77,82,1.23,1.30,97.090,336.938,0.793718,0.000000,NaN,-2.74,-0.548,-0.364964,0.697232,0.5,-0.197232,0.13,0.026,125.769231,0.309486,0.3,-0.009486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16615,2024-03-20,Eua Ncaa,22:10,Boise State,Colorado,2.10,1.73,138.5,1.83,1.83,0.0,0.00,0.00,z5Xw2grK,0.476190,0.578035,0.546448,0.546448,0.054225,145.742,71.288211,0.489140,1.8,1.643168,0.912871,123.42,1.910,0.854634,0.447453,23.0,120.404,32.446144,0.269477,2.4,1.341641,0.559017,119.68,1.714,0.350400,0.204434,33.0,66,68,1.87,1.76,114.062,128.532,0.136621,0.000000,NaN,-1.10,-0.220,-5.000000,0.684950,0.6,-0.084950,2.02,0.404,1.806931,0.579644,0.5,-0.079644
16616,2024-03-20,Eua Ncaa,19:40,Montana State,Grambling St.,1.61,2.42,134.5,1.91,1.91,-4.5,2.00,3.45,6mO9dX5s,0.621118,0.413223,0.523560,0.523560,0.034341,296.132,218.295570,0.737156,2.4,1.341641,0.559017,261.80,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16498,14:00,República Tcheca Nbl,Decin,Usti n. Labem,1.63,Back Home
16533,13:30,Finlândia Korisliiga Feminina,Forssan Alku F,Kouvottaret F,2.15,Back Home
16562,08:35,China Cba,Beijing,Shanghai,1.78,Back Home
16563,15:00,Bósnia E Herzegovina Prvenstvo Bih,Mrkonjic Grad,Spars Ilidza,1.69,Back Home
16571,14:00,Eslováquia Extraliga,BKM Lucenec,Handlova,2.02,Back Home
